# PURPOSE OF THIS NOTEBOOK

This is V2 of the data manipulation for the BERT model. Instead of trying to predict the exact ONET code, I will be using it to predict the major group the job belongs too. Then again try to predict the job code it belongs too once it predicts the major group it belongs too. (**SOURCE ONET DATA HERE**)

In [7]:
import pandas as pd
import numpy as np

In [8]:
# Start by loading in the data. 
onet_df = pd.read_csv("../Data/Updated_ONET_Alt_Titles.csv")
base_onet_df = pd.read_csv("../Data/Occupation Data.csv")

# Filter out any columns we don't need. 
onet_filtered = onet_df[['O*NET-SOC Code', 'Reported Job Title']]

# Show the filtered data
print(onet_filtered.shape)
onet_filtered.head()

(62306, 2)


,O*NET-SOC Code,Reported Job Title
0,11-1011.00,Chief Diversity Officer (CDO)
1,11-1011.00,Chief Executive Officer (CEO)
2,11-1011.00,Chief Financial Officer (CFO)
3,11-1011.00,Chief Nursing Officer
4,11-1011.00,Chief Operating Officer (COO)


In [9]:
# remove the military onet codes. 
base_onet_df = base_onet_df[~base_onet_df['O*NET-SOC Code'].str[:2].isin(['55'])]
onet_filtered = onet_filtered[~onet_filtered['O*NET-SOC Code'].str[:2].isin(['55'])]

## Start to strip the codes except for the first two, the major group. 

In [10]:
base_onet_df['O*NET-SOC Group'] = base_onet_df['O*NET-SOC Code'].str[:2]
onet_filtered['O*NET-SOC Group'] = onet_filtered['O*NET-SOC Code'].str[:2]

## Dummify the data and grab the labels each group belongs too

In [11]:
# Dummify the data. 
dummy_onets = pd.get_dummies(onet_filtered['O*NET-SOC Group'])

dummy_onets['Reported Job'] = onet_filtered['Reported Job Title']

In [12]:
dummy_onets.head()

,11,13,15,17,19,21,23,25,27,29,...,37,39,41,43,45,47,49,51,53,Reported Job
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Chief Diversity Officer (CDO)
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Chief Executive Officer (CEO)
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Chief Financial Officer (CFO)
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Chief Nursing Officer
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Chief Operating Officer (COO)


In [13]:
base_onet_df

,O*NET-SOC Code,Title,O*NET-SOC Group
0,11-1011.00,Chief Executives,11
1,11-1011.03,Chief Sustainability Officers,11
2,11-1021.00,General and Operations Managers,11
3,11-1031.00,Legislators,11
4,11-2011.00,Advertising and Promotions Managers,11
...,...,...,...
992,53-7072.00,"Pump Operators, Except Wellhead Pumpers",53
993,53-7073.00,Wellhead Pumpers,53
994,53-7081.00,Refuse and Recyclable Material Collectors,53
995,53-7121.00,"Tank Car, Truck, and Ship Loaders",53


In [15]:
# Do the same to the base ONET codes 
base_onet_df = base_onet_df[['O*NET-SOC Group', 'Title']]

dummy_base = pd.get_dummies(base_onet_df['O*NET-SOC Group'])

In [16]:
dummy_base['Reported Job'] = base_onet_df['Title']

In [17]:
dummy_base.head()

,11,13,15,17,19,21,23,25,27,29,...,37,39,41,43,45,47,49,51,53,Reported Job
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Chief Executives
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Chief Sustainability Officers
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,General and Operations Managers
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Legislators
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Advertising and Promotions Managers


In [18]:
# Add in default onet titles 
dummy_onets = pd.concat([dummy_base, dummy_onets], ignore_index=True)

# replace NaNs with False
dummy_onets.fillna(False, inplace=True)

In [19]:
# Seperate the labels and the reported jobs
label_df = dummy_onets.loc[:,dummy_onets.columns != 'Reported Job']
reported_df = dummy_onets.loc[:, dummy_onets.columns == 'Reported Job']

In [20]:
# Check that we isloated just the labels 
label_df.head()

,11,13,15,17,19,21,23,25,27,29,...,35,37,39,41,43,45,47,49,51,53
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [21]:
# export the label dataframe 
label_df.to_csv('../Data/label_df.csv', index=False)

In [22]:
# Check that we islated the reported jobs 
reported_df.head()

,Reported Job
0,Chief Executives
1,Chief Sustainability Officers
2,General and Operations Managers
3,Legislators
4,Advertising and Promotions Managers


In [23]:
# Lastly take the labels and change them to booleans
label_df = label_df.astype(int)

In [24]:
# Confirm that there are 0 null values in the label or reported jobs data 
print(label_df.isna().sum().sum())
print(reported_df.isna().sum().sum())

0
0


In [25]:
# Turn the labels into lists 
label_df['Label'] = label_df.values.tolist()

In [26]:
# Do a train test split on the data. 
def tt_split(df):
    ''' 1. Define the final dataframes that we need     
        2. Create a for loop that will iterate through all of the rows and do a 70/30 split on the data. 
        3. Append the new data to the final dataframes
        4. Reset the index of the final dataframes 
        5. Export the final dataframes. 
        STRETCH GOAL: take this function and convert the dataframe equations to numpy array equations, will be much faster.  ''' 

    # Define the final dataframes 
    test_df = pd.DataFrame(columns=['Reported_Jobs', 'Label'])
    train_df = pd.DataFrame(columns=['Reported_Jobs', 'Label'])

    # Grab all the columns apart from the final reported job title
    label_df = df.drop(columns=['Reported_Jobs', 'Label'])
    label_list = label_df.columns.to_list()
    
    for onet in label_list:
        filter_df = df[df[onet] == 1]
        temp_train_df = filter_df.sample(frac=.7,random_state=150)
        temp_test_df = filter_df.drop(temp_train_df.index).reset_index(drop=True)
        temp_train_df.reset_index(inplace=True, drop=True)

        # Append the new data to the final train/test dataframes 
        train_df = pd.concat([train_df, temp_train_df], ignore_index=True)
        test_df = pd.concat([test_df, temp_test_df], ignore_index=True)

    train_df.reset_index(drop=True, inplace=True)
    test_df.reset_index(drop=True, inplace=True)
    
    train_df = train_df[['Reported_Jobs', 'Label']]
    test_df = test_df[['Reported_Jobs', 'Label']]

    return train_df, test_df

In [27]:
# Create the final dataframe to export
label_df['Reported_Jobs'] = reported_df['Reported Job']


In [28]:
train_df, test_df = tt_split(label_df)

In [29]:
train_df.shape

(43869, 2)

In [30]:
test_df.shape

(18802, 2)

In [31]:
train_df

,Reported_Jobs,Label
0,"Farmers, Ranchers, and Other Agricultural Mana...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Supply Chain Planning Manager,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Sales Vice President (Sales VP),"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Special Programs Director,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Offshore Wind Operations Manager,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
43864,Sailors and Marine Oilers,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
43865,Dinkey Engine Fireman,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
43866,Airveyor Operator,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
43867,Process Operator,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


## Next Steps 

Instead of labeling out of over 1000 different options I'll thin the labels down to just the number of the major codes and try again. I would also need to see if I can extract skills from the major codes.

In [32]:
# Export the Training and Testing data. 
train_df.to_csv('../Data/Training_Data.csv', index=False)
test_df.to_csv('../Data/TestingData.csv', index=False)

## Next Steps 

From here we take the data that was set up for the model and split it into a train/test/validation. We will proceed to train and evaluate the model and proceed to start building the script to extract job titles out of resumes, then build another script to auto generate analytics to see how likely the applicant is a good fit and finally host all of this in an API. 